Install packages and import them

In [ ]:
pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Save the url of the website to scrap

In [159]:
url="https://www.f1technical.net/f1db/cars/years/1956"

In [160]:
response=requests.get(url)
soup=BeautifulSoup(response.text,"html.parser")
content=soup.find_all("div", {"class": "content"})
cars=content[2].findAll("tr")

In [161]:
cars[15].findAll("td")[0].a

<a href="/f1db/cars/87/kurtis-kraft-500c?sid=6eddfb88a1eb834897667d014b62a6e4">Kurtis Kraft 500C</a>

In [162]:
cars_list=[]
for i in range(1,len(cars)):
  car={}
  car_td=cars[i].findAll("td")
  if car_td[0].a==None: # Fill All the missing information with "Unknown" 
    car["Chassis"]="Unknown"
  else:
    car["Chassis"]=car_td[0].text

  if car_td[1].img==None:
    car["Country"]="Unknown"
  else:
    car["Country"]=car_td[1].img.get('title')
  
  if car_td[2].a==None:
    car["Team"]="Unknown"
  else:
    car["Team"]=car_td[2].text
  cars_list.append(car)

In [163]:
#cars_list

Convert the list to a Data Frame

In [164]:
df=pd.DataFrame(cars_list,columns=["Chassis","Country","Team"])
df

,Chassis,Country,Team
0,BRM P25,united kingdom,British Racing Motors
1,Bugatti T251,Unknown,Automobiles Bugatti
2,Connaught B,Unknown,Connaught Engineering
3,Cooper T23,united kingdom,Cooper Car Company
4,Epperly,united states,Epperly
5,Ferrari 500,italy,Scuderia Ferrari SpA
6,Ferrari 555,italy,Scuderia Ferrari SpA
7,Ferrari D50,italy,Scuderia Ferrari SpA
8,Gordini T16,Unknown,Equipe Gordini
9,Gordini T32,Unknown,Equipe Gordini
